In [ ]:
!pip3 install ktrain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 43.2 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 53.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.7/263.7 kB 14.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 23.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for ktrain: filename=ktrain-0.33.2-py3-none-any.whl size=25313841 sha256=8059a47c51c99617fa1fc242b8086b3d91917cd56a131fe567c084f150c70575
  Stored in directory: /root/.cache/pip/wheels/9c/e3/69/9942fefeb57744f93462fd91a8296f88f7dbf01aead387

In [ ]:
!pip install nltk

In [ ]:
import pandas as pd
import numpy as np

import ktrain
from ktrain import text
import nltk
from sklearn.model_selection import train_test_split,KFold
from sklearn.metrics import accuracy_score

In [ ]:
from torch.utils.data import TensorDataset,RandomSampler,SequentialSampler
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import spacy
import re,string,unicodedata

In [ ]:
path = '/kaggle/input/d/sriyar26/sexism-dataset/train_all_tasks.csv'

data_train = pd.read_csv(path)

In [ ]:
path1 = '/kaggle/input/d/sriyar26/sexism-dataset/datasets/dev_task_c_entries.csv'

data_test = pd.read_csv(path1)

In [ ]:
dftest=pd.read_csv('/kaggle/input/d/sriyar26/sexism-dataset/test_task_c_entries/test_task_c_entries.csv')# 

In [ ]:
data_train

,rewire_id,text,label_sexist,label_category,label_vector
0,sexism2022_english-7358,"Damn, this writing was pretty chaotic",not sexist,none,none
1,sexism2022_english-2367,"Yeah, and apparently a bunch of misogynistic v...",not sexist,none,none
2,sexism2022_english-3073,How the FUCK is this woman still an MP!!!???,not sexist,none,none
3,sexism2022_english-14895,Understand. Know you're right. At same time I ...,not sexist,none,none
4,sexism2022_english-4118,Surprized they didn't stop and rape some women,not sexist,none,none
...,...,...,...,...,...
13995,sexism2022_english-14185,complexes like the 'nice chicks' that go after...,sexist,2. derogation,2.1 descriptive attacks
13996,sexism2022_english-18629,"""GRAPHIC Germany - Muslim ""refugee"" stabbing h...",not sexist,none,none
13997,sexism2022_english-3320,Lol I imagine there would be simps that are li...,not sexist,none,none
13998,sexism2022_english-13309,"It's not, the girls I go on dates with don't k...",not sexist,none,none


In [ ]:
data_test.head(2)

,rewire_id,text
0,sexism2022_english-739,Every woman wants to be a model. It's codeword...
1,sexism2022_english-10787,Poor dude hasn't realized you dont share emoti...


In [ ]:
data_train['label_vector'].value_counts()

none                                                               10602
2.1 descriptive attacks                                              717
2.2 aggressive and emotive attacks                                   673
3.1 casual use of gendered slurs, profanities, and insults           637
3.2 immutable gender differences and gender stereotypes              417
4.2 supporting systemic discrimination against women as a group      258
1.2 incitement and encouragement of harm                             254
2.3 dehumanising attacks & overt sexual objectification              200
4.1 supporting mistreatment of individual women                       75
3.3 backhanded gendered compliments                                   64
1.1 threats of harm                                                   56
3.4 condescending explanations or unwelcome advice                    47
Name: label_vector, dtype: int64

In [ ]:
index1 = data_train[ (data_train['label_vector'] == 'none')  ].index
data_train.drop(index1 , inplace=True)

In [ ]:
classes = ['1.1 threats of harm','1.2 incitement and encouragement of harm','2.1 descriptive attacks','2.2 aggressive and emotive attacks','2.3 dehumanising attacks & overt sexual objectification','3.1 casual use of gendered slurs, profanities, and insults','3.2 immutable gender differences and gender stereotypes','3.3 backhanded gendered compliments','3.4 condescending explanations or unwelcome advice','4.1 supporting mistreatment of individual women','4.2 supporting systemic discrimination against women as a group']


In [ ]:
data_train=data_train.drop('label_sexist', axis='columns')

In [ ]:
data_train=data_train.drop('label_category', axis='columns')

In [ ]:
data_train=data_train.drop('rewire_id', axis='columns')

In [ ]:
data_train.head()

,text,label_vector
8,"just the inside of your wallet: cash, bank car...",2.3 dehumanising attacks & overt sexual object...
11,They get the initial chance because they are a...,2.1 descriptive attacks
17,ugly chicks A pussy is a pussy.,2.3 dehumanising attacks & overt sexual object...
18,fuck her off with a nine mm this is what you g...,1.2 incitement and encouragement of harm
19,How about I talk shit to an idiot like you? Ar...,"3.1 casual use of gendered slurs, profanities,..."


In [ ]:
data_train['label_vector'].unique()

array(['2.3 dehumanising attacks & overt sexual objectification',
       '2.1 descriptive attacks',
       '1.2 incitement and encouragement of harm',
       '3.1 casual use of gendered slurs, profanities, and insults',
       '4.2 supporting systemic discrimination against women as a group',
       '2.2 aggressive and emotive attacks',
       '3.2 immutable gender differences and gender stereotypes',
       '3.4 condescending explanations or unwelcome advice',
       '3.3 backhanded gendered compliments',
       '4.1 supporting mistreatment of individual women',
       '1.1 threats of harm'], dtype=object)

In [ ]:
# Import label encoder
from sklearn import preprocessing

# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()

# Encode labels in column 'species'.
data_train['label_vector']= label_encoder.fit_transform(data_train['label_vector'])

data_train['label_vector'].unique()


array([ 4,  2,  1,  5, 10,  3,  6,  8,  7,  9,  0])

In [ ]:
import re
def clean_text(text):

    text = re.sub(r"@[A-Za-z0-9]+", ' ', text)
    text = re.sub(r"https?://[A-Za-z0-9./]+", ' ', text)
    text = re.sub(r"[^a-zA-z.!?'0-9]", ' ', text)
    text = re.sub('\t', ' ',  text)
    text = re.sub(r" +", ' ', text)
    text=re.sub(r'[^\w\s]','',text)
    text=text.replace("[URL]","")
    text=text.replace("[USER]","")
    text=re.sub(r"[â€™ºðŸ‡˜Žµ±¤£‘Œ”œ]","",text)
    return text

In [ ]:
data_train['text']= data_train['text'].apply(clean_text)
data_test['text']= data_test['text'].apply(clean_text)
dftest['text']= dftest['text'].apply(clean_text)

In [ ]:
#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

#Removing the noisy text
def denoise_text(text):
    
    text = remove_between_square_brackets(text)
    return text

In [ ]:
data_train['text']=data_train['text'].apply(denoise_text)
data_test['text']= data_test['text'].apply(denoise_text)
dftest['text']= dftest['text'].apply(denoise_text)

In [ ]:
kf = KFold(n_splits=5, random_state=42, shuffle=True)

In [ ]:
models = []
val_accs = []

In [ ]:
MODEL_NAME='roberta-base'#'microsoft/deberta-v3-base'

In [ ]:
t  = text.Transformer(MODEL_NAME, maxlen=500,class_names=[0, 1,2,3,4,5,6,7,8,9,10])

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/627M [00:00<?, ?B/s]

In [ ]:
def transformer_cv():
    predictions,accs=[],[]
    data = data_train[['text', 'label_vector']]
    for train_index, val_index in kf.split(data):
        train,val=data.iloc[train_index],data.iloc[val_index]
#         print(train)
#         print(val)
        x_train=train.text.tolist()
        x_val=val.text.tolist()
        y_train=train.label_vector.tolist()
        y_val=val.label_vector.tolist()
#         print(x_train)
#         print(y_train)
        trn = t.preprocess_train(x_train, y_train)
        val = t.preprocess_test(x_val, y_val)
        model = t.get_classifier()
        learner = ktrain.get_learner(model, train_data=trn, batch_size=6)
        learner.fit_onecycle(2e-5, 4)
        predictor = ktrain.get_predictor(learner.model, t)
        pred=predictor.predict(x_val)
        acc=accuracy_score(y_val,pred)
        print('acc',acc)
        accs.append(acc)
        models.append(learner.model)
        val_acc = learner.validate(class_names=t.get_classes(), val_data=val)[1]
        val_accs.append(val_acc)
        
    return accs,val_accs

In [ ]:
print( transformer_cv() )

In [ ]:
# get the best performing model and its validation accuracy
best_model_idx = val_accs.index(max(val_accs))
best_model = models[best_model_idx]
best_val_acc = val_accs[best_model_idx]
# save the best performing model
predictor = ktrain.get_predictor(best_model, preproc=t)
predictor.get_classes()
predictor.save('roberta')